In [ ]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokens = tokenizer.encode(text, add_special_tokens=False)

In [ ]:
context_length = 10
batch_size = 64

In [ ]:
num_batches = len(tokens) // (batch_size * context_length)
tokens = tokens[:num_batches * batch_size * context_length]

In [ ]:
import torch

input_ids = torch.tensor(tokens).view(-1, context_length)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
print(input_ids.shape)
dataset = TensorDataset(input_ids)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
from labml_nn.transformers.LoRA.GPT2 import GPTModel

model = GPTModel()

In [ ]:
optimizer = Adam(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()

model.eval()
epochs = 3
for epoch in range(epochs):
    for batch in dataloader:
        inputs = batch[0]
        labels = inputs.clone()
        
        outputs = model(inputs)
        
        shift_logits = outputs[..., :-1, :]
        shift_labels = labels[..., 1:]
        
        loss = criterion(shift_logits.reshape(-1, shift_logits.size(-1)), shift_labels.reshape(-1))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')
        break

print("Training complete.")